# Project 4 Fletcher
## Part 4 Extracting for Named Entity

### Explored frontier technology news in decades from New York Times API. Applied unsupervised machine learning methods to cluster different topics in tech news and compared the popular fields in time series.


Technology Trend in New York Times: rest API | NLP | LDA | Word2Vec | PCA | Clustering | MongoDB
<br>
• Summarized top topics from frontier technology news in 1945-2017 using New York Times API.
<br>
• Extracted public relationship network of aimed company from mass data.


In [13]:
import pandas as pd
import numpy as np
import nltk
from time import sleep
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.chunk import ne_chunk
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.decomposition import LatentDirichletAllocation
from pymongo import MongoClient
import json
from copy import deepcopy
import matplotlib.pyplot as plt
from pycorenlp import StanfordCoreNLP

%matplotlib inline

# Loading Data

In [4]:
import pickle
with open('tech1945_2017.plk','rb') as f:
    t=pickle.load(f)

In [5]:
lpdocs_yearly = []
for i in range(1945,2018):
    lpdocs_yearly.append([doc['lead_paragraph'] for doc in t[i] if doc['lead_paragraph']!=None])

In [6]:
# lpdocs_yearly is list of lists of strings
print(type(lpdocs_yearly), type(lpdocs_yearly[0]), type(lpdocs_yearly[0][0]))

<class 'list'> <class 'list'> <class 'str'>


# Name Entity Extraction

# encoding text as UTF-8 
lp2016 = lpdocs_yearly[-2]

In [8]:
lp2016 = lpdocs_yearly[-2]

In [9]:
lp2016 = [lp.encode(encoding='UTF-8',errors='strict') for lp in lp2016]

In [14]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [15]:
lp = lpdocs_yearly[-1]

lp = [l.encode(encoding='UTF-8',errors='strict') for l in lp]

relations = []
for doc in lp:

    nes = nlp.semgrex(doc, pattern='{tag: NNP}', filter=False)
    for s in nes['sentences']:
        del s['length']
        nnp_objs = s.values()

        # skip sentences that contain no NNP
        if len(nnp_objs) == 0:
            continue

        # sort nnp by their begin position
        nnp_objs = sorted(nnp_objs, key=lambda k: k['begin'])

        nnps = []
        i = 0
        while i < len(nnp_objs):

            curr_nnp = nnp_objs[i]['text']

            # combine consecutive NNP
            j = i
            while j < len(nnp_objs)-1:
                if nnp_objs[j]['end'] == nnp_objs[j+1]['begin']:
                    curr_nnp += ' ' + nnp_objs[j+1]['text']
                    j += 1
                else:
                    break
            if len(curr_nnp) > 0:
                nnps.append(curr_nnp) 

            i = j+1

        if len(nnps) > 0:
            relations.append(nnps)
with open('relations2017.plk','wb') as f:
    pickle.dump(relations,f)


In [64]:
relations[:10]

[['Google', 'Facebook'],
 ['Politwoops', 'API'],
 ['Thursday', 'Duchenne'],
 ['Rebuild', 'Mailbox'],
 ['Adam Bosworth'],
 ['Firefox'],
 ['MANICAS', 'Peter T.'],
 ['Peter T. Manicas',
  'Queens College',
  'City University',
  'New York',
  'University',
  'Hawaii',
  'Manoa',
  'December',
  'Honolulu'],
 ['Kenoe Kauanoe', 'Ti', 'Manicas', 'Teddy', 'Theodore'],
 ['Tautasi']]